In [0]:
#Get Data
! wget https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv
! wget https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv

--2019-02-28 08:50:01--  https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ggl9krhh6dcwhhz/train.csv [following]
--2019-02-28 08:50:06--  https://www.dropbox.com/s/raw/ggl9krhh6dcwhhz/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc94f0a5235555133383d1f427bb.dl.dropboxusercontent.com/cd/0/inline/AcLsFTHMnEJbeilJcCD85RciH966pSJZpgVNpa4t_kkEbFBDSny-VE0RRXri9l7re6Ca0HR7dMMBtceUibMqOqIw4Aqr1IukZBxdkSdNm0MdJEsxdhkH9ieDBmQpf7gIYdg/file# [following]
--2019-02-28 08:50:07--  https://uc94f0a5235555133383d1f427bb.dl.dropboxusercontent.com/cd/0/inline/AcLsFTHMnEJbeilJcCD85RciH966pSJZpgVNpa4t_kkEbFBDSny-VE0RRXri9l7re6Ca0HR7dMMBtceUibMqOqIw4Aqr1IukZBxdkSdNm0MdJEsxdhkH9ieDBmQ

In [0]:
! ls -ls train.csv

67196 -rw-r--r-- 1 root root 68802655 Feb 28 08:50 train.csv


In [0]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')

training = train['comment_text']
testing = test['comment_text']

#combining data for fitting the Vectorizer
combined_text = pd.concat([training, testing])


In [0]:
#to vectorize word
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(combined_text)
train_words = word_vectorizer.transform(training)
test_words = word_vectorizer.transform(testing)



In [0]:
#to vectorize character
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(combined_text)
train_char = char_vectorizer.transform(training)
test_char = char_vectorizer.transform(testing)

In [0]:
train_features = hstack([train_words, train_char])  #combine word and char using hstack for training
test_features = hstack([test_words, test_char]) #combine word and char using hstack for testing

In [0]:
train_target = train['toxic'] #set training target

##Define Model

In [0]:
#define model
classifier = LogisticRegression(C=0.1, solver='sag')
classifier.fit(train_features, train_target)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
#ROC using cross validation on train data
scores = []
cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
scores.append(cv_score)
print('Total CV score is {}'.format(np.mean(scores)))


Total CV score is 0.9697993945465266


##Validation on traning Data (test.csv)

In [0]:
#genrating final sheet and prediction
result = pd.DataFrame.from_dict({'id': test['id']})  
result['toxic'] = classifier.predict_proba(test_features)[:, 1] #probablity on test features storing in result considering the only class present


In [0]:
test_target = test['toxic'] #setting testing data target

In [0]:
#roc on testing data 
from sklearn.metrics import roc_auc_score
roc_auc_score(test['toxic'],result['toxic']) #calculating accuracy 

0.9583474623098701

In [0]:
#result is having predicted pridictions and corresponding id's
result.describe()

,toxic
count,63978.000000
mean,0.119019
std,0.206544
min,0.000731
25%,0.017120
50%,0.036361
75%,0.101262
max,0.999999


In [0]:
result.sample(10)

,id,toxic
14152,3870a1e0385c7ead,0.015347
48098,bf86717e77ba41a1,0.042650
58562,e9fdaa62c75a2811,0.105222
41245,a4752e851bc5baf0,0.610957
5589,16a57e4e1dfd393f,0.004309
53935,d75b917419b34675,0.255467
12904,338c7da30dc114bd,0.016381
51998,cf786039b4fbfe43,0.054556
60239,f09d8d8bfbbc9b7e,0.055665
42403,a91716e82c37c289,0.229725
